In [17]:
import os
import json
from PIL import Image
from io import BytesIO
import base64
from datetime import datetime
from geopy.geocoders import Nominatim

In [18]:
# Set the folder containing your Google Takeout data, i.e. photos and json files
TAKEOUT_FOLDER = " "

In [19]:

# Set up the geocoder
geolocator = Nominatim(user_agent="photo_index_script")

# Function to generate a thumbnail image encoded in base64 for embedding in HTML
def generate_thumbnail(image_path):
    with Image.open(image_path) as img:
        img.thumbnail((150, 150))  # Resize to 150x150
        buffered = BytesIO()
        img.save(buffered, format="JPEG")
        thumbnail_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")
        return thumbnail_base64

#function to get suburb name
def get_detailed_location(latitude, longitude):
    try:
        location = geolocator.reverse((latitude, longitude), language='en', exactly_one=True)
        if location:
            address = location.raw.get('address', {})
            suburb = address.get('suburb', '')
            town = address.get('town', '') or address.get('city', '')
            city = address.get('city', '')
            village = address.get('village', '')
            
            # Priority order: suburb -> town -> city -> village
            if suburb:
                return suburb + " / " + town
            elif town:
                return town + " / " + city
            elif city:
                return city
            elif village:
                return village
            else:
                return "Unknown"
    except Exception as e:
        print(f"Error fetching detailed location for coordinates {latitude}, {longitude}: {e}")
    return "Unknown"

# Prepare the HTML table content
html_content = """
<html>
<head>
    <style>
        table { width: 100%; border-collapse: collapse; }
        th, td { padding: 8px; text-align: left; border: 1px solid #ddd; }
        img { width: 150px; height: 150px; }
    </style>
</head>
<body>
    <h2>Photo Index</h2>
    <table>
        <tr>
            <th>Thumbnail</th>
            <th>File Name</th>
            <th>Date Taken</th>
            <th>Town</th>
        </tr>
"""

# Walk through the Google Takeout folder to find photos and their metadata
photo_data = []
for root, _, files in os.walk(TAKEOUT_FOLDER):
    for file in files:
        # Check if the file is a JSON metadata file (assumed pattern: .json)
        if file.endswith('.json'):
            json_path = os.path.join(root, file)
            with open(json_path, 'r', encoding='utf-8') as f:
                metadata = json.load(f)
                
            # Extract file name from metadata (e.g., "PXL_...jpg")
            title = metadata.get("title")
            if title:
                # Get photo taken timestamp
                timestamp = metadata.get("photoTakenTime", {}).get("timestamp")
                if timestamp:
                    # Convert timestamp to a readable date
                    photo_date = datetime.utcfromtimestamp(int(timestamp)).strftime('%Y-%m-%d %H:%M:%S')
                    
                    # Get the full path of the photo
                    photo_path = os.path.join(root, title)
                    if os.path.exists(photo_path):
                        # Generate thumbnail
                        thumbnail_base64 = generate_thumbnail(photo_path)
                        
                        # Extract geoData if available
                        geo_data = metadata.get("geoData", {})
                        latitude = geo_data.get("latitude")
                        longitude = geo_data.get("longitude")
                        town_name = "Unknown"
                        
                        if latitude and longitude:
                            town_name = get_detailed_location(latitude, longitude)
                        
                        # Store photo data for later use
                        photo_data.append({
                            "thumbnail": thumbnail_base64,
                            "file_name": title,
                            "date_taken": photo_date,
                            "town": town_name,
                            "photo_path": photo_path
                        })

# Sort the photo data by date taken (chronologically)
photo_data.sort(key=lambda x: x["date_taken"])

# Add photo data to the HTML content
for photo in photo_data:
    html_content += f"""
    <tr>
        <td><img src="data:image/jpeg;base64,{photo['thumbnail']}"></td>
        <td>{photo['file_name']}</td>
        <td>{photo['date_taken']}</td>
        <td>{photo['town']}</td>
    </tr>
    """

# Close the HTML tags
html_content += """
    </table>
</body>
</html>
"""

# Write the HTML content to a file
output_path = "photo_index_with_town.html"
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"HTML file generated with town names: {output_path}")


HTML file generated with town names: photo_index_with_town.html
